In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# An example of a Low Liquidity Trade

In [3]:
import requests
import s3fs
import pandas as pd
import re
import humps
import sqlalchemy as sa
from datetime import timedelta, datetime
import datetime as dt
import pendulum


import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import MapTickerSymbols
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

import etl_moc

In [4]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [5]:
# See a list of tsx moc data
moc_file_lst = get_moc_file_lst.run("tsx-moc")
moc_file_lst

['tsx-moc/moc_20200324.csv',
 'tsx-moc/moc_20200325.csv',
 'tsx-moc/moc_20200326.csv',
 'tsx-moc/moc_20200327.csv',
 'tsx-moc/moc_20200330.csv',
 'tsx-moc/moc_tsx_20200331.csv',
 'tsx-moc/moc_tsx_20200401.csv',
 'tsx-moc/moc_tsx_20200402.csv',
 'tsx-moc/moc_tsx_20200403.csv',
 'tsx-moc/moc_tsx_20200406.csv',
 'tsx-moc/moc_tsx_20200407.csv',
 'tsx-moc/moc_tsx_20200409.csv',
 'tsx-moc/moc_tsx_20200413.csv',
 'tsx-moc/moc_tsx_20200414.csv',
 'tsx-moc/moc_tsx_20200415.csv',
 'tsx-moc/moc_tsx_20200416.csv']

In [6]:
# Get all the files
data_flpth_lst = map(lambda flnm:  f"s3://{flnm}", moc_file_lst)
df_lst = [pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False) for data_flpth in list(data_flpth_lst)]
tsx_moc_df = pd.concat(df_lst , ignore_index=True)
tsx_moc_df.loc[tsx_moc_df["Symbol"]=="", "Symbol"] = "NA"

In [7]:
tsx_moc_df[tsx_moc_df["Symbol"].isna()==True]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


In [8]:
tsx_moc_df["Symbol"].nunique()

825

## Create a key table
Map TSX symbols to Yahoo Symbols

In [9]:
yhooMap =  MapTickerSymbols()
moc_key_df = yhooMap.run(tsx_moc_df)
moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

In [10]:
moc_key_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7073 entries, 0 to 7072
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   tsx_symbol                 7073 non-null   object        
 1   imbalance_side             7073 non-null   object        
 2   imbalance_size             7073 non-null   int64         
 3   imbalance_reference_price  7073 non-null   float64       
 4   moc_date                   7073 non-null   datetime64[ns]
 5   yahoo_symbol               7073 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 331.7+ KB


In [11]:
moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

## Get daily features
(from yahoo)

#### Price

In [12]:
eod_price_df = etl_moc.get_eod_price_data.run(moc_key_df)

2020-03-24
[*********************100%***********************]  387 of 387 completed

1 Failed download:
- BL-UN.TO: No data found for this date range, symbol may be delisted
2020-03-25
[*********************100%***********************]  417 of 417 completed

7 Failed downloads:
- WIR-R.TO: No data found for this date range, symbol may be delisted
- GEC-UN.TO: No data found for this date range, symbol may be delisted
- EF-UN.TO: No data found for this date range, symbol may be delisted
- RA-UN.TO: No data found for this date range, symbol may be delisted
- BL-UN.TO: No data found for this date range, symbol may be delisted
- GDG-UN.TO: No data found for this date range, symbol may be delisted
- CM-PY.TO: No data found for this date range, symbol may be delisted
2020-03-26
[*********************100%***********************]  407 of 407 completed

4 Failed downloads:
- GEC-UN.TO: No data found for this date range, symbol may be delisted
- EF-UN.TO: No data found for this date range, symbol

#### Other Attributes
(float, short interest)

In [13]:
eod_info_df = etl_moc.get_sym_info.run(moc_key_df)

825


INFO:root:Error getting info from yahoo for sym BL-UN.TO
INFO:root:Error getting info from yahoo for sym CEF.TO
INFO:root:Error getting info from yahoo for sym CM-PO.TO
INFO:root:Error getting info from yahoo for sym CM-PP.TO
INFO:root:Error getting info from yahoo for sym CM-PR.TO
INFO:root:Error getting info from yahoo for sym CM-PS.TO
INFO:root:Error getting info from yahoo for sym CM-PT.TO
INFO:root:Error getting info from yahoo for sym RA-UN.TO
INFO:root:Error getting info from yahoo for sym WIR-R.TO
INFO:root:Error getting info from yahoo for sym CPX-PI.TO
INFO:root:Error getting info from yahoo for sym TA-PH.TO
INFO:root:Error getting info from yahoo for sym W-PK.TO
INFO:root:Error getting info from yahoo for sym TRP-PD.TO
INFO:root:Error getting info from yahoo for sym TRP-PE.TO
INFO:root:Error getting info from yahoo for sym BRF-PA.TO
INFO:root:Error getting info from yahoo for sym TRP-PH.TO
INFO:root:Error getting info from yahoo for sym BAM-PF-A.TO
INFO:root:Error getting in

## Get indtraday features
(from yahoo)

In [14]:
intraday_df = etl_moc.get_1min_ohlc.run(moc_key_df)

2020-03-24
[*********************100%***********************]  387 of 387 completed
2020-03-25
[*********************100%***********************]  417 of 417 completed

1 Failed download:
- MBN.TO: No data found for this date range, symbol may be delisted
2020-03-26
[*********************100%***********************]  407 of 407 completed

1 Failed download:
- MBN.TO: No data found for this date range, symbol may be delisted
2020-03-27
[*********************100%***********************]  407 of 407 completed

2 Failed downloads:
- RA-UN.TO: No data found for this date range, symbol may be delisted
- MBN.TO: No data found for this date range, symbol may be delisted
2020-03-30
[*********************100%***********************]  406 of 406 completed

1 Failed download:
- MBN.TO: No data found for this date range, symbol may be delisted
2020-03-31
[*********************100%***********************]  687 of 687 completed

33 Failed downloads:
- ENB-PF-G.TO: No data found, symbol may be deliste

## Create MOC table

In [15]:
moc_df = etl_moc.build_moc_data.run(intraday_df, eod_price_df, eod_info_df)

In [ ]:
moc_df

## Write to Db

In [16]:
# 1. Eod
index_clmn_lst = ["moc_date", "yahoo_symbol"]

In [18]:
num_rows_ins = etl_moc.df_to_db.run(intraday_df, tbl_name="intraday_prices", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 2. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_price_df, tbl_name="eod_prices", idx_clmn_lst=index_clmn_lst)

# 3. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_info_df, tbl_name="eod_info", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 4. Write to db
num_rows_ins = etl_moc.df_to_db.run(moc_df, tbl_name="daily_moc", idx_clmn_lst=index_clmn_lst)

In [ ]:
from prefect import task, Flow
from prefect.tasks.secrets import EnvVarSecret

@task
def print_value(x):
    print(x)

with Flow("Example") as flow:
    secret = EnvVarSecret("moc_pgdb_conn")
    print_value(secret)

In [ ]:
flow.run()

In [ ]:
import prefect
from prefect.client import Secret

In [ ]:
prefect.context.secrets["AWS_CREDENTIALS"] = 

In [ ]:
s = Secret("hello")

In [ ]:
s.exists()

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
import boto3
session = boto3.session.Session()
client = session.client(
        service_name='secretsmanager'
    )
get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )

In [ ]:
secret = get_secret_value_response['SecretString']

In [ ]:
db_creds = get_db_creds()
type(db_creds)

In [ ]:
db_creds

In [ ]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:
# https://aws.amazon.com/developers/getting-started/ruby/

# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "moc-pg-db"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    # Your code goes here. 

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")